# Learning rate: plateau

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torch
import torch.nn as nn
import multiprocessing
import torch.optim as optim
from torch.utils.data import DataLoader
from main import MNIST_dataset, MNIST_trainer

In [9]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu124
Device:  cuda


In [10]:
####################################################################
# DataLoader Class
####################################################################
train_dataset = MNIST_dataset(partition="train")
test_dataset = MNIST_dataset(partition="test")

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)
train_dataloader = DataLoader(
    train_dataset, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_dataset, batch_size, shuffle=False, num_workers=num_workers
)


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [11]:
####################################################################
# Neural Network Class
####################################################################


# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(
        self,
        sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, 10]],
        criterion=None,
    ):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()

        for i in range(len(sizes) - 1):
            dims = sizes[i]
            self.layers.append(nn.Linear(dims[0], dims[1]))
            self.layers.append(nn.BatchNorm1d(dims[1]))
            self.layers.append(nn.ReLU())

        dims = sizes[-1]
        self.classifier = nn.Linear(dims[0], dims[1])

        self.criterion = criterion

    def forward(self, x, y=None):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)

        if y != None:
            loss = self.criterion(x, y)
            return loss, x
        return x


####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(
    sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, num_classes]],
    criterion=criterion,
)
print(net)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Params: ", count_parameters(net))
# Training hyperparameters
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)


scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, "min", patience=5, threshold=0.001, cooldown=1, verbose=True
)
epochs = 75

trainer = MNIST_trainer(
    net,
    train_dataloader,
    test_dataloader,
    optimizer,
    criterion,
    epochs,
    device,
    scheduler,
    model_path="models/lr.pt",
)

Net(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)
Params:  2919434


In [12]:
####################################################################
# Training
####################################################################

trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:00<00:00, 134.57batch/s]

	LR:  0.01
[Epoch 1] Train Loss: 0.001693 - Test Loss: 0.000816 - Train Accuracy: 94.85% - Test Accuracy: 97.31%



Test 1: 100%|██████████| 100/100 [00:00<00:00, 118.67batch/s]


	LR:  0.01
[Epoch 2] Train Loss: 0.000522 - Test Loss: 0.000698 - Train Accuracy: 98.39% - Test Accuracy: 97.84%


...

	LR:  1.0000000000000004e-08
[Epoch 73] Train Loss: 0.000025 - Test Loss: 0.000501 - Train Accuracy: 99.98% - Test Accuracy: 98.59%



Test 73: 100%|██████████| 100/100 [00:00<00:00, 111.83batch/s]

	LR:  1.0000000000000004e-08
[Epoch 74] Train Loss: 0.000026 - Test Loss: 0.000503 - Train Accuracy: 99.99% - Test Accuracy: 98.53%



Test 74: 100%|██████████| 100/100 [00:00<00:00, 109.09batch/s]

	LR:  1.0000000000000004e-08
[Epoch 75] Train Loss: 0.000025 - Test Loss: 0.000500 - Train Accuracy: 99.99% - Test Accuracy: 98.56%

BEST TEST ACCURACY:  98.61  in epoch  45


In [13]:
####################################################################
# Load best weights
####################################################################

trainer.get_model()

/home/diego/code/RNA/mnist_tests/main.py:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  


Test 74: 100%|██████████| 100/100 [00:01<00:00, 79.77batch/s]

Final best acc:  98.61
